# Convert clipper-formatted data to mfa-formatted data
This file walks through three pre-processing steps:
1. Cleaning the data,
2. Converting the data for tool use, and
3. Generating phone-level alignments.



Dry-run to check for any errors in the Clipper files we have. Sometimes there's a filename mismatch, a missing character name, missing transcript file, or similar.

In [11]:
!(cd ../src; python -m datapipes \
    --input /home/celestia/data/orig-samples `# clipper-formatted directory` \
    --output /home/celestia/data/mfa-inputs `# mfa-formatted directory` \
    --delta `# ignore files already processed` \
    --dry-run `# don't create any output files`)

If there are any errors, make sure to fix them and re-run the above command. Repeat until there are no errors, then run the next command to generate the mfa-formatted data.

In [ ]:
!(cd ../src; python -m datapipes \
    --input /home/celestia/data/orig-samples \
    --output /home/celestia/data/mfa-inputs \
    --delta)

The below two files are known to cause problems for montreal-forced-aligned. Get rid of them before running the forced aligner.

In [ ]:
# this one screws with montreal-forced-aligner for some reason
!rm /home/celestia/data/mfa-inputs/Pinkie-Pie/S5_s5e19_00_02_05_Pinkie_Neutral__A.wav
!rm /home/celestia/data/mfa-inputs/Pinkie-Pie/S5_s5e19_00_02_05_Pinkie_Neutral__A.textgrid

Run montreal-forced-aligner with the following command to generate phone-level alignments.

In [ ]:
%%bash
ls data/mfa-inputs | while read line;
do
    mkdir /home/celestia/data/mfa-alignments/$line

    yes n | mfa_align -v `# continue even with an incomplete dictionary` \
        /home/celestia/data/mfa-inputs/$line `# input directory` \
        /opt/mfa/pronunciations_dicts/english.dict.txt \
        /opt/mfa/pretrained_models/english.zip \
        /home/celestia/data/mfa-alignments/$line `# output directory`
done